In [76]:
import numpy as np
import altair as alt
import pandas as pd
from html.parser import HTMLParser
import os
import yaml
from bs4 import BeautifulSoup
from bs4.element import NavigableString

import geopandas as gpd
import pickle
import re

```
html_file = "./state-territorial-universities.html"
os.path.isfile(html_file)
```

```
def make_list(tag):
    elements = []
    if tag.name=='ul':
        for st in tag.children:
            if st.name == 'li' and st.a:
                all_ul = [] if st is NavigableString else st.find_all('ul')
                if len(all_ul)==0:
                    elements.append(str(st.a.contents[0]))
                elif st.a:
                    elements.append({str(st.a.contents[0]): make_list(all_ul[0])})
    return elements
```

```
elements = {}
with open(html_file) as file:
    curr_state = ''
    soup = BeautifulSoup(file, "html.parser").contents[-1]
    for tag in soup.body.contents:
        if tag.name=='h2' and tag.contents[0].name=='span':
            curr_state = str(tag.contents[0].contents[0])
        elif tag.name=='ul':
            elements[curr_state] = make_list(tag)
elements
with open('universities.yaml', 'w') as out_file:
    yaml.dump(elements, out_file)
```

In [186]:
cc_data = pd.read_csv('data_processed.csv')

if False:
    cc_data['lon'] = cc_data['geometry'].map(lambda p: float(re.findall(r'-?\d+[:.]\d+\w', str(p))[0]))
    cc_data['lat'] = cc_data['geometry'].map(lambda p: float(re.findall(r'-?\d+[:.]\d+\w', str(p))[1]))
cc_data

,unitid,name,city,state,Undergraduate Program,Graduate Program,Enrollment Profile,Undergraduate Profile,Size & Setting,Basic,full_state,address,lon,lat
0,999911,Air University,Montgomery,AL,Baccalaureate/Associates Colleges,Research Doctoral Single program Other,Very high undergraduate,Two-year higher part-time,Four-year large primarily nonresidential,Master's Colleges & Universities Larger Programs,Alabama,"Air University, Sycamore Street, 36112, Sycamo...",-86.345485,32.385264
1,100654,Alabama A & M University,Normal,AL,Professions plus arts & sciences some graduate...,Research Doctoral Professional-dominant,High undergraduate,Four-year full-time inclusive lower transfer-in,Four-year medium highly residential,Master's Colleges & Universities Larger Programs,Alabama,Alabama Agricultural and Mechanical University...,-86.568423,34.783895
2,100724,Alabama State University,Montgomery,AL,Balanced arts & sciences/professions some grad...,Research Doctoral Professional-dominant,Very high undergraduate,Four-year full-time inclusive lower transfer-in,Four-year medium highly residential,Doctoral Universities Doctoral/Professional Un...,Alabama,"Alabama State University, 915, South Jackson S...",-86.293551,32.363350
3,100858,Auburn University,Auburn,AL,Professions plus arts & sciences high graduate...,Research Doctoral Comprehensive programs with ...,High undergraduate,Four-year full-time more selective lower trans...,Four-year large primarily nonresidential,Doctoral Universities Very High Research Activity,Alabama,"Auburn University, 36849, Auburn, AL, United S...",-85.495949,32.591988
4,100830,Auburn University at Montgomery,Montgomery,AL,Professions plus arts & sciences some graduate...,Postbaccalaureate Comprehensive programs,High undergraduate,Four-year medium full-time lower transfer-in h...,Four-year medium primarily residential,Master's Colleges & Universities Larger Programs,Alabama,"University Church of Christ, Atlanta Highway, ...",-86.219345,32.382527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,240480,University of Wisconsin-Stevens Point,Stevens Point,WI,Balanced arts & sciences/professions some grad...,Postbaccalaureate Other-dominant with Arts & S...,Very high undergraduate,Four-year full-time inclusive lower transfer-in,Four-year medium primarily residential,Master's Colleges & Universities Medium Programs,Wisconsin,"University of Wisconsin–Stevens Point, Reserve...",-89.569467,44.531640
466,240417,University of Wisconsin-Stout,Menomonie,WI,Professions focus some graduate coexistence,Postbaccalaureate Education-dominant with Arts...,Very high undergraduate,Four-year full-time selective lower transfer-in,Four-year medium primarily residential,Master's Colleges & Universities Larger Programs,Wisconsin,"University of Wisconsin-Stout, 712, Broadway S...",-91.926516,44.871548
467,240426,University of Wisconsin-Superior,Superior,WI,Balanced arts & sciences/professions some grad...,Postbaccalaureate Education-dominant with Arts...,Very high undergraduate,Four-year medium full-time lower transfer-in h...,Four-year small primarily residential,Master's Colleges & Universities Medium Programs,Wisconsin,"University District, 54880, Superior, Wisconsi...",-92.092294,46.720951
468,240189,University of Wisconsin-Whitewater,Whitewater,WI,Professions plus arts & sciences some graduate...,Postbaccalaureate Comprehensive programs,Very high undergraduate,Four-year full-time inclusive lower transfer-in,Four-year large primarily residential,Master's Colleges & Universities Larger Programs,Wisconsin,"University of Wisconsin-Whitewater, 800, West ...",-88.743666,42.844360


In [78]:
from geopandas.tools import geocode

if False:
    cc_data = pd.read_csv('~/Downloads/cc_download.csv')

    geos = geocode(cc_data['name'], provider='photon', user_agent='unc-comp-790')
    cc_data = cc_data.join(geos)
    cc_data.head()

In [79]:
not_found = cc_data.loc[cc_data.geometry.apply(lambda p: p.is_empty)]
not_found.head()

AttributeError: 'DataFrame' object has no attribute 'geometry'

In [184]:
# cc_data.to_csv('data_processed.csv')

In [7]:
from geopandas.tools import geocode

geos = geocode(not_found['name'], provider='photon', user_agent='unc-comp-790')
geos

,geometry,address
18,POINT EMPTY,None
178,POINT (0.00970 51.54433),"University of East London (Stratford Campus), ..."
219,POINT (77.23261 28.66446),Indira Gandhi Delhi Technical University for W...
220,POINT (-90.34263 33.51126),"Mississippi Valley State University, County Ro..."
236,POINT (72.59375 23.10587),"Gujarat Technological University, 382424, Ahme..."
248,POINT (151.23466 -33.90624),"University of New South Wales Randwick Campus,..."
261,POINT (-74.17903 40.74213),"New Jersey Institute of Technology, University..."
270,POINT EMPTY,None
276,POINT (-73.89016 40.87334),"Bedford Park Boulevard–Lehman College, Bedford..."
278,POINT (-73.82520 40.73700),"Main Street & CUNY Queens College, Main Street..."


In [152]:
# cc_data['geometry'][not_found.index] = not_found['geometry']

In [9]:
cc_data['lon'] = cc_data['geometry'].map(lambda p: None if p.is_empty else p.x)
cc_data['lat'] = cc_data['geometry'].map(lambda p: None if p.is_empty else p.y)
cc_data[['lon','lat']].head()

,lon,lat
0,-86.345485,32.385264
1,-86.568423,34.783895
2,-86.345485,32.385264
3,-85.495949,32.591988
4,-86.345485,32.385264


In [187]:
wrong_state = cc_data.loc[~cc_data.apply(lambda r: r['city'] in r['address'] or r['name'] in r['address'], axis=1)]
wrong_state[['name','city','address']]

,name,city,address
67,University of Colorado Denver/Anschutz Medical...,Denver,University of Colorado Anschutz Medical Campus...
187,University of Massachusetts-Dartmouth,North Dartmouth,"University of Massachusetts Dartmouth, 285, Ol..."
357,University of Puerto Rico-Mayaguez,Mayaguez,Universidad de Puerto Rico - Recinto Universit...


In [179]:
import requests
import time

url = "https://us1.locationiq.com/v1/search"

responses = []

for i in wrong_state.index:
    data = {
        'key': 'pk.4390cfb99f9a354407e60d9cb64cd356',
        'street': wrong_state['name'][i],
        'format': 'json',
        'city': wrong_state['city'][i]
    }
    response = requests.get(url, params=data)
    responses.append(response.json())
    time.sleep(0.5)
responses

[{'error': 'Unable to geocode'},
 {'error': 'Unable to geocode'},
 {'error': 'Unable to geocode'},
 [{'place_id': '210638515',
   'licence': 'https://locationiq.com/attribution',
   'osm_type': 'way',
   'osm_id': '463534916',
   'boundingbox': ['41.9308182', '41.943627', '-88.7970677', '-88.7498739'],
   'lat': '41.937093399999995',
   'lon': '-88.78267841443035',
   'display_name': 'Northern Illinois University, 1425, West Lincoln Highway, DeKalb, DeKalb Township, DeKalb County, Illinois, 60115, USA',
   'class': 'amenity',
   'type': 'university',
   'importance': 0.8906416873565544,
   'icon': 'https://locationiq.org/static/images/mapicons/education_university.p.20.png'}],
 {'error': 'Unable to geocode'},
 {'error': 'Unable to geocode'},
 {'error': 'Unable to geocode'},
 [{'place_id': '332405759081',
   'osm_type': 'node',
   'osm_id': '358269095',
   'licence': 'https://locationiq.com/attribution',
   'lat': '41.62959',
   'lon': '-71.006186',
   'display_name': 'University of Mas

In [180]:
for i, r in enumerate(responses):
    if 'error' in r:
        print(cc_data['name'][wrong_state.index[i]])
    else:
        cc_data['address'][wrong_state.index[i]] = r[0]['display_name']
        cc_data['lat'][wrong_state.index[i]] = float(r[0]['lat'])
        cc_data['lon'][wrong_state.index[i]] = float(r[0]['lon'])

Colorado State University-Global Campus
University of Colorado Denver/Anschutz Medical Campus
The University of West Florida
University of Southern Indiana
Northwestern State University of Louisiana
University of Maryland Global Campus
University of Minnesota-Twin Cities
Mississippi Valley State University
Ohio University-Main Campus
Wright State University-Main Campus
Virginia State University
The Evergreen State College


C:\Users\Sam Ehrenstein\AppData\Local\Temp\ipykernel_28796\685767713.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_data['address'][wrong_state.index[i]] = r[0]['display_name']
C:\Users\Sam Ehrenstein\AppData\Local\Temp\ipykernel_28796\685767713.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_data['lat'][wrong_state.index[i]] = float(r[0]['lat'])
C:\Users\Sam Ehrenstein\AppData\Local\Temp\ipykernel_28796\685767713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [188]:
if 'geometry' in cc_data:
    cc_data.pop('geometry')
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, feature='states')

map = alt.Chart(states).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('albersUsa').properties(
        width=800,
        height=600
    )
points = alt.Chart(cc_data).mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    color=alt.Color('Basic:N'),
    tooltip=['name:N','lon:Q']
).project("albersUsa")
map + points

alt.LayerChart(...)

In [16]:
cc_data.to_csv('data_processed.csv')

In [17]:
geos

,geometry,address
18,POINT EMPTY,None
178,POINT (0.00970 51.54433),"University of East London (Stratford Campus), ..."
219,POINT (77.23261 28.66446),Indira Gandhi Delhi Technical University for W...
220,POINT (-90.34263 33.51126),"Mississippi Valley State University, County Ro..."
236,POINT (72.59375 23.10587),"Gujarat Technological University, 382424, Ahme..."
248,POINT (151.23466 -33.90624),"University of New South Wales Randwick Campus,..."
261,POINT (-74.17903 40.74213),"New Jersey Institute of Technology, University..."
270,POINT EMPTY,None
276,POINT (-73.89016 40.87334),"Bedford Park Boulevard–Lehman College, Bedford..."
278,POINT (-73.82520 40.73700),"Main Street & CUNY Queens College, Main Street..."
